In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions

# Step 1: View the list of reports available

In [2]:
report_lst = helper_functions.get_available_reports()

In [3]:
#Turn it into a list to see all of them
list(report_lst['ReportName'])

['_Current Employee Deductions and Benefits',
 '2013-03-05T21:47:40.188Z',
 '2020-10-26T18:25:32.678Z',
 '2021 Allocation Report (1)',
 '401k Allocation Report V1',
 '401k Allocation Report V2',
 '401k Allocation Report V2_UKG',
 '401k Allocation Report V3_UKG',
 '401k Allocation Report V3_UKG v2',
 '401k Allocation Report V3_UKG v4',
 '5 Level Supervisors',
 'Active EE_DD List',
 'Allocated Hours FTE by Date',
 'Allocated Hours FTE by Month',
 'Annual Earnings',
 'Audit Report with Time vs Pay Matrix Tables',
 'Comp Project V3',
 'Configuration Validation',
 'Counts by Location',
 'Current Employee Deductions and Benefits',
 "David's Report",
 "David's Report V2",
 'Deduction list',
 'Deduction Pay History By Deduction For EmployeesUKG',
 'Deductions in Arrears V2',
 'Deferred Comp Wages and Hours_UKG',
 'Dependents by Employee and Plan test',
 'Document Category Report',
 'DSHS_New Hires',
 'Earning Codes Table',
 'Earnings Pay History By Earning Code For Employees_ULTI',
 'Earnings_

In [4]:
#Get the precheck deduction report pathname
report_lst[report_lst['ReportName'].str.contains('pre', regex=True, case=False)]

,ReportName,ReportPath
61,Pre-Check Deduction,/content/folder[@name='zzzCompany Folders']/fo...
62,Pre-Check Earnings,/content/folder[@name='zzzCompany Folders']/fo...
63,Pre-Check Master,/content/folder[@name='zzzCompany Folders']/fo...
64,Pre-Check Payroll Register,/content/folder[@name='zzzCompany Folders']/fo...
65,Pre-Check Tax,/content/folder[@name='zzzCompany Folders']/fo...
66,Prenote,/content/folder[@name='zzzCompany Folders']/fo...


In [5]:
report_path = report_lst[report_lst['ReportName'] == 'Pre-Check Tax']['ReportPath'].loc[65]
report_path

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Pre-Check Tax']"

## Step 2: Retrieve the report

In [6]:
df = helper_functions.retrieve_report(report_path) 
df

c:\UKG\Reports as a service\helper_functions.py:367: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('{', '')
c:\UKG\Reports as a service\helper_functions.py:368: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('}', '')
c:\UKG\Reports as a service\helper_functions.py:369: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('[', '')
c:\UKG\Reports as a service\helper_functions.py:370: FutureWarning: The defa

,period_control_date,employee_number,stub_description,tax_code,tax_description,tax_group_code,tax_type,employer_tax,current_taxable_gross,current_tax_amount
0,2022-12-09T00:00:00,001003291,Employee Medicare,USMEDEE,Employee Medicare,TG,MED,N,29746,431.32
1,2022-12-09T00:00:00,001003291,Employer Medicare,USMEDER,Employer Medicare,TG,MED,Y,29746,431.32
2,2022-12-09T00:00:00,001003291,Federal Income Tax,USFIT,Federal Income Tax,TG,FIT,N,29746,8181.29
3,2022-12-09T00:00:00,001003291,Federal Unemployment Tax,USFUTA,Federal Unemployment Tax,TG,FUT,Y,29746,0
4,2022-12-09T00:00:00,001003291,Social Security Employee Tax,USSOCEE,Social Security Employee Tax,TG,SOC,N,29746,1638.16
...,...,...,...,...,...,...,...,...,...,...
75,2022-12-09T00:00:00,001008586,WA Vol Paid Fam Leave-ER pd EE,WAVPFLEP,WA Vol Paid Fam Leave-ER pd EE,TG,SDI,Y,0,0
76,2022-12-09T00:00:00,001008586,WA Vol Paid Med Leave- EE,WAVPMLEE,WA Vol Paid Med Leave- EE,TG,SDI,N,0,0
77,2022-12-09T00:00:00,001008586,WA Vol Paid Med Leave- ER,WAVPMLER,WA Vol Paid Med Leave- ER,TG,SDI,Y,0,0
78,2022-12-09T00:00:00,001008586,WA Workforce Training Employer,WAWTFER,WA Workforce Training Employer,TG,SUI,Y,0,0


In [7]:
#Clean the columns
df['period_control_date'] = df['period_control_date'].str.replace('T', ' ')

#View the results
df

,period_control_date,employee_number,stub_description,tax_code,tax_description,tax_group_code,tax_type,employer_tax,current_taxable_gross,current_tax_amount
0,2022-12-09 00:00:00,001003291,Employee Medicare,USMEDEE,Employee Medicare,TG,MED,N,29746,431.32
1,2022-12-09 00:00:00,001003291,Employer Medicare,USMEDER,Employer Medicare,TG,MED,Y,29746,431.32
2,2022-12-09 00:00:00,001003291,Federal Income Tax,USFIT,Federal Income Tax,TG,FIT,N,29746,8181.29
3,2022-12-09 00:00:00,001003291,Federal Unemployment Tax,USFUTA,Federal Unemployment Tax,TG,FUT,Y,29746,0
4,2022-12-09 00:00:00,001003291,Social Security Employee Tax,USSOCEE,Social Security Employee Tax,TG,SOC,N,29746,1638.16
...,...,...,...,...,...,...,...,...,...,...
75,2022-12-09 00:00:00,001008586,WA Vol Paid Fam Leave-ER pd EE,WAVPFLEP,WA Vol Paid Fam Leave-ER pd EE,TG,SDI,Y,0,0
76,2022-12-09 00:00:00,001008586,WA Vol Paid Med Leave- EE,WAVPMLEE,WA Vol Paid Med Leave- EE,TG,SDI,N,0,0
77,2022-12-09 00:00:00,001008586,WA Vol Paid Med Leave- ER,WAVPMLER,WA Vol Paid Med Leave- ER,TG,SDI,Y,0,0
78,2022-12-09 00:00:00,001008586,WA Workforce Training Employer,WAWTFER,WA Workforce Training Employer,TG,SUI,Y,0,0


# Step 3: Export the report as a CSV

In [8]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [9]:
helper_functions.create_sql_table_v2('STG_PreCheck_Tax')

# Step 5: Close the API connection

In [10]:
helper_functions.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'